In [2]:
# ============================================================================
# بخش 1: تنظیمات اولیه و ایمپورت کتابخانه‌های ضروری
# ============================================================================

import sys
import os
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'torch'

In [3]:
# تنظیم مسیرها
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, 'data')
print(f"دایرکتوری جاری: {current_dir}")
print(f"دایرکتوری داده: {data_dir}")


دایرکتوری جاری: e:\AI_Proj\Uni\icwsm-2024-news-criticism\src
دایرکتوری داده: e:\AI_Proj\Uni\icwsm-2024-news-criticism\src\data


In [4]:
# بررسی وجود فایل‌های داده
files = os.listdir(data_dir)
print("\nفایل‌های موجود در دایرکتوری data:")
for file in files:
    file_path = os.path.join(data_dir, file)
    file_size = os.path.getsize(file_path) / (1024*1024)  # حجم به مگابایت
    print(f"  - {file} ({file_size:.2f} MB)")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'e:\\AI_Proj\\Uni\\icwsm-2024-news-criticism\\src\\data'

In [5]:

# تنظیم seed برای reproducibility
def set_seed(seed=42):
    """تنظیم seed برای reproducibility در تمام کتابخانه‌ها"""
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(42)

NameError: name 'torch' is not defined

In [6]:
# بررسی GPU
print(f"\nاطلاعات سخت‌افزاری:")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("از CPU استفاده می‌شود")


اطلاعات سخت‌افزاری:


NameError: name 'torch' is not defined

In [7]:
# ============================================================================
# بخش 2: بارگذاری داده‌های واقعی
# ============================================================================

print("\n" + "="*60)
print("بخش 2: بارگذاری داده‌های واقعی")
print("="*60)

# 2.1 بارگذاری داده‌های خبری
print("\n1. بارگذاری داده‌های خبری (news_df.csv)...")
news_df_path = os.path.join(data_dir, 'news_df.csv')
try:
    news_df = pd.read_csv(news_df_path)
    print(f"   ✅ داده‌های خبری بارگذاری شد: {news_df.shape} سطر و {news_df.shape[1]} ستون")
    print(f"   📊 ستون‌ها: {list(news_df.columns)}")
    
    # نمایش نمونه‌ای از داده‌های خبری
    print(f"\n   نمونه‌ای از داده‌های خبری:")
    print(f"   تعداد منابع خبری: {len(news_df)}")
    print(f"   رنج امتیازات حزبی: {news_df['Partisan Score'].min()} تا {news_df['Partisan Score'].max()}")
    
    # نمایش 5 منبع خبری اول
    print(f"\n   اولین 5 منبع خبری:")
    for i in range(min(5, len(news_df))):
        print(f"   {i+1}. {news_df.iloc[i]['Source']} - امتیاز: {news_df.iloc[i]['Partisan Score']}")
        
except Exception as e:
    print(f"   ❌ خطا در بارگذاری داده‌های خبری: {e}")
    news_df = None


بخش 2: بارگذاری داده‌های واقعی

1. بارگذاری داده‌های خبری (news_df.csv)...
   ❌ خطا در بارگذاری داده‌های خبری: [Errno 2] No such file or directory: 'e:\\AI_Proj\\Uni\\icwsm-2024-news-criticism\\src\\data\\news_df.csv'


In [8]:
# 2.2 بارگذاری داده‌های حاشیه‌نویسی شده
print("\n2. بارگذاری داده‌های حاشیه‌نویسی شده (annotated_data_anonymized.jsonl)...")
annotated_path = os.path.join(data_dir, 'annotated_data_anonymized.jsonl')
try:
    # خواندن فایل JSONL
    annotated_data = []
    with open(annotated_path, 'r', encoding='utf-8') as f:
        for line in f:
            annotated_data.append(json.loads(line))
    
    # تبدیل به DataFrame
    annotated_df = pd.DataFrame(annotated_data)
    print(f"   ✅ داده‌های حاشیه‌نویسی شده بارگذاری شد: {annotated_df.shape} سطر")
    print(f"   📊 ستون‌ها: {list(annotated_df.columns)}")
    
    # نمایش آماری از داده‌ها
    print(f"\n   آمار داده‌های حاشیه‌نویسی شده:")
    print(f"   - تعداد توییت‌ها: {len(annotated_df)}")
    if 'label' in annotated_df.columns:
        print(f"   - توزیع برچسب‌ها:")
        label_counts = annotated_df['label'].value_counts()
        for label, count in label_counts.items():
            print(f"     * برچسب {label}: {count} ({count/len(annotated_df)*100:.1f}%)")
            
except Exception as e:
    print(f"   ❌ خطا در بارگذاری داده‌های حاشیه‌نویسی شده: {e}")
    annotated_df = None


2. بارگذاری داده‌های حاشیه‌نویسی شده (annotated_data_anonymized.jsonl)...
   ❌ خطا در بارگذاری داده‌های حاشیه‌نویسی شده: [Errno 2] No such file or directory: 'e:\\AI_Proj\\Uni\\icwsm-2024-news-criticism\\src\\data\\annotated_data_anonymized.jsonl'


In [9]:
# 2.3 بارگذاری داده‌های اصلی برای تحلیل
print("\n3. بارگذاری داده‌های اصلی برای تحلیل (data_for_analysis_anonymized.jsonl)...")
analysis_path = os.path.join(data_dir, 'data_for_analysis_anonymized.jsonl')
try:
    # برای فایل‌های بزرگ، نمونه‌ای از داده‌ها را می‌خوانیم
    print("   در حال خواندن نمونه‌ای از داده‌ها (این ممکن است چند لحظه طول بکشد)...")
    
    # ابتدا تعداد خطوط را می‌شماریم
    with open(analysis_path, 'r', encoding='utf-8') as f:
        num_lines = sum(1 for _ in f)
    print(f"   تعداد کل خطوط در فایل: {num_lines:,}")
    
    # خواندن نمونه‌ای از داده‌ها (اولین 1000 خط)
    analysis_data = []
    with open(analysis_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i < 1000:  # فقط 1000 خط اول
                analysis_data.append(json.loads(line))
            else:
                break
    
    # تبدیل به DataFrame
    analysis_df = pd.DataFrame(analysis_data)
    print(f"   ✅ نمونه‌ای از داده‌های تحلیل بارگذاری شد: {analysis_df.shape} سطر")
    print(f"   📊 ستون‌ها: {list(analysis_df.columns)}")
    
    # نمایش آماری
    print(f"\n   آمار نمونه داده‌ها:")
    print(f"   - تعداد کاربران منحصربفرد: {analysis_df['user'].nunique() if 'user' in analysis_df.columns else 'N/A'}")
    
except Exception as e:
    print(f"   ❌ خطا در بارگذاری داده‌های تحلیل: {e}")
    analysis_df = None


3. بارگذاری داده‌های اصلی برای تحلیل (data_for_analysis_anonymized.jsonl)...
   در حال خواندن نمونه‌ای از داده‌ها (این ممکن است چند لحظه طول بکشد)...
   ❌ خطا در بارگذاری داده‌های تحلیل: [Errno 2] No such file or directory: 'e:\\AI_Proj\\Uni\\icwsm-2024-news-criticism\\src\\data\\data_for_analysis_anonymized.jsonl'


In [10]:
# ============================================================================
# بخش 3: بارگذاری ماژول‌های پروژه
# ============================================================================

print("\n" + "="*60)
print("بخش 3: بارگذاری ماژول‌های پروژه")
print("="*60)

# لیست ماژول‌های مورد نیاز
modules = [
    'preprocessing_utils',
    'feature_extractors', 
    'analysis_utils',
    'torch_datasets',
    'models',
    'model_utils',
    'labelling_functions',
    'inference_utils',
    'plot_utils'
]


بخش 3: بارگذاری ماژول‌های پروژه


In [11]:
# بارگذاری ماژول‌ها
loaded_modules = {}
for module_name in modules:
    try:
        # ساخت مسیر فایل
        module_file = f"{module_name}.py"
        
        if os.path.exists(module_file):
            print(f"   🔄 در حال بارگذاری {module_name}...")
            
            # خواندن محتوای فایل
            with open(module_file, 'r', encoding='utf-8') as f:
                module_content = f.read()
            
            # اجرای ماژول
            exec(module_content, globals())
            loaded_modules[module_name] = True
            print(f"   ✅ {module_name} بارگذاری شد")
        else:
            print(f"   ⚠️ فایل {module_file} یافت نشد")
            loaded_modules[module_name] = False
            
    except Exception as e:
        print(f"   ❌ خطا در بارگذاری {module_name}: {str(e)[:100]}...")
        loaded_modules[module_name] = False

print(f"\nنتایج بارگذاری ماژول‌ها:")
for module_name, status in loaded_modules.items():
    status_symbol = "✅" if status else "❌"
    print(f"   {status_symbol} {module_name}")

   🔄 در حال بارگذاری preprocessing_utils...
   ❌ خطا در بارگذاری preprocessing_utils: cannot import name 'PS_LF' from partially initialized module 'labelling_functions' (most likely due ...
   🔄 در حال بارگذاری feature_extractors...
   ✅ feature_extractors بارگذاری شد
   🔄 در حال بارگذاری analysis_utils...
   ✅ analysis_utils بارگذاری شد
   🔄 در حال بارگذاری torch_datasets...
   ❌ خطا در بارگذاری torch_datasets: cannot import name 'translation_init' from partially initialized module 'preprocessing_utils' (most ...
   🔄 در حال بارگذاری models...
   ❌ خطا در بارگذاری models: No module named 'transformers'...
   🔄 در حال بارگذاری model_utils...
   ❌ خطا در بارگذاری model_utils: No module named 'sklearn'...
   🔄 در حال بارگذاری labelling_functions...
   ❌ خطا در بارگذاری labelling_functions: cannot import name 'translation_init' from partially initialized module 'preprocessing_utils' (most ...
   🔄 در حال بارگذاری inference_utils...
   ❌ خطا در بارگذاری inference_utils: No module named 'to

<string>:754: SyntaxWarning: invalid escape sequence '\p'
<string>:756: SyntaxWarning: invalid escape sequence '\p'
<string>:758: SyntaxWarning: invalid escape sequence '\p'


In [12]:
# ============================================================================
# بخش 4: پیش‌پردازش داده‌ها با داده‌های واقعی
# ============================================================================

print("\n" + "="*60)
print("بخش 4: پیش‌پردازش داده‌ها")
print("="*60)

if loaded_modules['preprocessing_utils'] and analysis_df is not None:
    print("انجام پیش‌پردازش روی داده‌های واقعی...")
    
    try:
        # آماده‌سازی داده‌ها برای پیش‌پردازش
        # ابتدا ساختار داده‌ها را بررسی می‌کنیم
        print(f"\nساختار داده‌های ورودی:")
        print(f"- ستون‌ها: {list(analysis_df.columns)}")
        print(f"- تعداد نمونه: {len(analysis_df)}")
        
        # بررسی ستون‌های ضروری
        required_columns = ['text', 'user', 'tweet_id', 'matched_partisans', 'matched_sources']
        missing_columns = [col for col in required_columns if col not in analysis_df.columns]
        
        if missing_columns:
            print(f"⚠️ ستون‌های ضروری وجود ندارند: {missing_columns}")
            print("تلاش برای استفاده از ستون‌های موجود...")
            
            # اگر ستون matched_partisans وجود ندارد، یک ستون نمونه ایجاد می‌کنیم
            if 'matched_partisans' not in analysis_df.columns:
                print("ایجاد ستون matched_partisans نمونه...")
                analysis_df['matched_partisans'] = np.random.choice([-2, -3, 2, 3], len(analysis_df))
            
            # اگر ستون matched_sources وجود ندارد، یک ستون نمونه ایجاد می‌کنیم
            if 'matched_sources' not in analysis_df.columns:
                print("ایجاد ستون matched_sources نمونه...")
                analysis_df['matched_sources'] = [['CNN'] if i % 2 == 0 else ['FoxNews'] for i in range(len(analysis_df))]
        
        # افزودن ستون‌های لازم اگر وجود ندارند
        if 'tweet_type' not in analysis_df.columns:
            analysis_df['tweet_type'] = 'replied_to'
        
        if 'referenced_text' not in analysis_df.columns:
            analysis_df['referenced_text'] = [[] for _ in range(len(analysis_df))]
        
        if 'matched_mentions' not in analysis_df.columns:
            analysis_df['matched_mentions'] = analysis_df['matched_sources']
        
        if 'matched_urls' not in analysis_df.columns:
            analysis_df['matched_urls'] = [[] for _ in range(len(analysis_df))]
        
        if 'tweet_public_metrics' not in analysis_df.columns:
            analysis_df['tweet_public_metrics'] = [{'retweet_count': 0, 'reply_count': 1, 'like_count': 7, 'quote_count': 0} 
                                                  for _ in range(len(analysis_df))]
        
        # نمونه‌ای از داده‌ها برای پیش‌پردازش (برای سرعت)
        sample_size = min(500, len(analysis_df))
        sample_data = analysis_df.head(sample_size).copy()
        
        print(f"\nپیش‌پردازش {sample_size} نمونه از داده‌ها...")
        
        # ایجاد شناساگر ریتوییت مستقیم
        if news_df is not None:
            drt_identifier = DRIdentifier(news_df)
            
            # شناسایی ریتوییت‌های مستقیم
            sample_data = identify_direct_retweets(sample_data, drt_identifier)
            print(f"   - شناسایی ریتوییت‌های مستقیم انجام شد")
            
            # استخراج هندل‌های توییتر
            twitter_handles = news_df.explode("Twitter Handle")
            twitter_handles = twitter_handles.loc[twitter_handles['Twitter Handle'].notna()]
            twitter_handles = twitter_handles["Twitter Handle"].tolist()
            
            # شناسایی زنجیره‌های پاسخ طولانی
            sample_data = identify_long_replychains(sample_data, twitter_handles)
            print(f"   - شناسایی زنجیره‌های پاسخ طولانی انجام شد")
        
        # حذف توییت‌های بدون mention
        sample_data = remove_non_mentions(sample_data)
        print(f"   - حذف توییت‌های بدون mention انجام شد")
        
        # دوختن متن توییت‌ها
        sample_data["stitched_text"] = sample_data.apply(lambda x: stitch_tweets(x), axis=1)
        print(f"   - دوختن متن توییت‌ها انجام شد")
        
        print(f"\n✅ پیش‌پردازش با موفقیت انجام شد")
        print(f"   - اندازه داده‌های پیش‌پردازش شده: {sample_data.shape}")
        print(f"   - تعداد ریتوییت‌های مستقیم: {sample_data['direct_retweet'].sum() if 'direct_retweet' in sample_data.columns else 0}")
        
        # ذخیره داده‌های پیش‌پردازش شده
        preprocessed_path = os.path.join(current_dir, 'preprocessed_sample.pkl')
        sample_data.to_pickle(preprocessed_path)
        print(f"   - داده‌های پیش‌پردازش شده در {preprocessed_path} ذخیره شد")
        
        preprocessed_data = sample_data
        
    except Exception as e:
        print(f"❌ خطا در پیش‌پردازش داده‌ها: {e}")
        preprocessed_data = None
else:
    print("⚠️ پیش‌پردازش داده‌ها امکان‌پذیر نیست")
    preprocessed_data = None


بخش 4: پیش‌پردازش داده‌ها
⚠️ پیش‌پردازش داده‌ها امکان‌پذیر نیست


In [13]:
# ============================================================================
# بخش 5: استخراج ویژگی‌ها
# ============================================================================

print("\n" + "="*60)
print("بخش 5: استخراج ویژگی‌ها")
print("="*60)

if loaded_modules['feature_extractors'] and preprocessed_data is not None and news_df is not None:
    print("استخراج ویژگی‌های کاربر از داده‌های پیش‌پردازش شده...")
    
    try:
        # ایجاد داده‌های سیاستمداران نمونه
        pol_data = {
            'user_name': ['politician1', 'politician2', 'politician3', 'politician4'],
            'party_name': ['Republican', 'Democrat', 'Republican', 'Democrat']
        }
        pol_df = pd.DataFrame(pol_data)
        
        # ایجاد extractor
        print("ایجاد استخراج‌کننده ویژگی‌های کاربر...")
        user_feats_extractor = ExtractUserFeats(
            news_df=news_df,
            pol_df=pol_df,
            following_path=data_dir  # مسیر موقت
        )
        
        # استخراج ویژگی‌ها برای هر کاربر
        print("استخراج ویژگی‌ها...")
        
        # گروه‌بندی بر اساس کاربر
        user_groups = list(preprocessed_data.groupby('user'))
        print(f"تعداد کاربران: {len(user_groups)}")
        
        # استخراج ویژگی‌ها برای اولین کاربر
        if user_groups:
            user_name, user_df = user_groups[0]
            print(f"\nاستخراج ویژگی‌ها برای کاربر: {user_name}")
            print(f"تعداد توییت‌های کاربر: {len(user_df)}")
            
            # استخراج ویژگی‌ها
            user_df_with_features = user_feats_extractor.extract(user_df)
            
            print(f"✅ ویژگی‌ها با موفقیت استخراج شدند")
            print(f"ستون‌های جدید اضافه شده:")
            new_columns = [col for col in user_df_with_features.columns if col not in preprocessed_data.columns]
            for col in new_columns:
                print(f"   - {col}")
            
            # نمایش نمونه‌ای از ویژگی‌های استخراج شده
            if 'partisan_dist' in user_df_with_features.columns:
                sample_dist = user_df_with_features.iloc[0]['partisan_dist']
                print(f"\nنمونه توزیع حزبی: {sample_dist}")
        
    except Exception as e:
        print(f"❌ خطا در استخراج ویژگی‌ها: {e}")
else:
    print("⚠️ استخراج ویژگی‌ها امکان‌پذیر نیست")


بخش 5: استخراج ویژگی‌ها
⚠️ استخراج ویژگی‌ها امکان‌پذیر نیست


In [14]:
# ============================================================================
# بخش 6: برچسب‌زنی داده‌ها
# ============================================================================

print("\n" + "="*60)
print("بخش 6: برچسب‌زنی داده‌ها با توابع برچسب‌زنی")
print("="*60)

if loaded_modules['labelling_functions'] and preprocessed_data is not None:
    print("اعمال توابع برچسب‌زنی بر داده‌ها...")
    
    try:
        # ایجاد نمونه‌ای از داده‌ها برای برچسب‌زنی
        labelling_data = preprocessed_data.copy()
        
        # ایجاد توابع برچسب‌زنی
        print("\nایجاد توابع برچسب‌زنی...")
        
        # تابع برچسب‌زنی بر اساس توزیع حزبی
        ps_lf = PS_LF(threshold=0.1, min_count=10)
        print(f"✅ تابع PS_LF ایجاد شد")
        
        # تابع برچسب‌زنی بر اساس ریتوییت مستقیم
        drt_lf = DRT_LF(threshold=0.15, min_count=25)
        print(f"✅ تابع DRT_LF ایجاد شد")
        
        # تابع برچسب‌زنی بر اساس دنبال کردن سیاستمداران
        if pol_df is not None:
            pol_lf = PolFol_LF(pol_df=pol_df, threshold=0.05, min_count=5)
            print(f"✅ تابع PolFol_LF ایجاد شد")
        
        # تابع برچسب‌زنی سطح توییت
        tweet_lf = TweetLevelLF()
        print(f"✅ تابع TweetLevelLF ایجاد شد")
        
        # اعمال توابع برچسب‌زنی (روی نمونه کوچک)
        sample_for_labelling = labelling_data.head(100).copy()
        
        print(f"\nاعمال توابع برچسب‌زنی بر {len(sample_for_labelling)} نمونه...")
        
        # اعمال تابع PS_LF
        sample_for_labelling = ps_lf.label(sample_for_labelling)
        print(f"   - تابع PS_LF اعمال شد")
        
        # اعمال تابع DRT_LF
        sample_for_labelling = drt_lf.label(sample_for_labelling)
        print(f"   - تابع DRT_LF اعمال شد")
        
        # اعمال تابع PolFol_LF
        if 'pol_lf' in locals():
            sample_for_labelling = pol_lf.label(sample_for_labelling)
            print(f"   - تابع PolFol_LF اعمال شد")
        
        # اعمال تابع TweetLevelLF
        sample_for_labelling = tweet_lf.label(sample_for_labelling)
        print(f"   - تابع TweetLevelLF اعمال شد")
        
        # بررسی ستون‌های برچسب ایجاد شده
        label_columns = [col for col in sample_for_labelling.columns if 'label' in col]
        print(f"\nستون‌های برچسب ایجاد شده: {label_columns}")
        
        # نمایش توزیع برچسب‌ها
        for col in label_columns:
            if col in sample_for_labelling.columns:
                value_counts = sample_for_labelling[col].value_counts()
                print(f"\nتوزیع برچسب‌ها برای {col}:")
                for value, count in value_counts.items():
                    print(f"   برچسب {value}: {count} نمونه")
        
        labelled_data = sample_for_labelling
        
    except Exception as e:
        print(f"❌ خطا در برچسب‌زنی داده‌ها: {e}")
        labelled_data = None
else:
    print("⚠️ برچسب‌زنی داده‌ها امکان‌پذیر نیست")
    labelled_data = None


بخش 6: برچسب‌زنی داده‌ها با توابع برچسب‌زنی
⚠️ برچسب‌زنی داده‌ها امکان‌پذیر نیست


In [15]:
# ============================================================================
# بخش 7: تحلیل داده‌های برچسب‌زده شده
# ============================================================================

print("\n" + "="*60)
print("بخش 7: تحلیل داده‌های برچسب‌زده شده")
print("="*60)

if loaded_modules['analysis_utils'] and labelled_data is not None:
    print("انجام تحلیل‌های آماری بر داده‌های برچسب‌زده شده...")
    
    try:
        # بررسی پوشش برچسب‌ها
        print("\n1. بررسی پوشش برچسب‌ها:")
        coverage_df = get_coverage(labelled_data)
        print(coverage_df.to_string())
        
        # بررسی آمار برچسب‌زنی
        print("\n2. آمار کلی برچسب‌زنی:")
        get_labelling_stats(labelled_data)
        
        # تحلیل توافق و تعارض بین توابع برچسب‌زنی
        if 'unanimous_label' in labelled_data.columns:
            print("\n3. تحلیل توافق بین توابع برچسب‌زنی:")
            
            # محاسبه درصد توافق
            agreement_stats = labelled_data['unanimous_label'].value_counts(normalize=True)
            print("توزیع برچسب‌های توافقی:")
            for label, percent in agreement_stats.items():
                print(f"   برچسب {label}: {percent*100:.1f}%")
            
            # شمارش تعارضات
            conflicts = (labelled_data['unanimous_label'] == -100).sum()
            total = len(labelled_data)
            print(f"\nتعداد تعارضات: {conflicts} از {total} ({conflicts/total*100:.1f}%)")
        
    except Exception as e:
        print(f"❌ خطا در تحلیل داده‌ها: {e}")
else:
    print("⚠️ تحلیل داده‌ها امکان‌پذیر نیست")


بخش 7: تحلیل داده‌های برچسب‌زده شده
⚠️ تحلیل داده‌ها امکان‌پذیر نیست


In [16]:
# ============================================================================
# بخش 8: ایجاد دیتاست‌های PyTorch
# ============================================================================

print("\n" + "="*60)
print("بخش 8: ایجاد دیتاست‌های PyTorch برای آموزش مدل")
print("="*60)

if loaded_modules['torch_datasets'] and labelled_data is not None and news_df is not None:
    print("ایجاد دیتاست‌های PyTorch از داده‌های برچسب‌زده شده...")
    
    try:
        # آماده‌سازی داده‌ها برای دیتاست
        # نیاز به ستون برچسب داریم
        if 'keyword_label' in labelled_data.columns:
            label_col = 'keyword_label'
        elif 'user_label' in labelled_data.columns:
            label_col = 'user_label'
        else:
            # ایجاد ستون برچسب نمونه
            print("ایجاد ستون برچسب نمونه...")
            labelled_data['sample_label'] = np.random.choice([0, 1], len(labelled_data))
            label_col = 'sample_label'
        
        # ایجاد دیتاست متن
        print(f"\n1. ایجاد TextDataset با برچسب {label_col}...")
        text_dataset = TextDataset(
            dataset_df=labelled_data,
            label_col=label_col,
            pre_trained="cardiff",
            max_length=50
        )
        
        print(f"   ✅ TextDataset ایجاد شد")
        print(f"   - تعداد نمونه‌ها: {len(text_dataset)}")
        
        # تست یک نمونه
        sample = text_dataset[0]
        print(f"   - شکل ویژگی‌های متن: {sample['feats'].shape}")
        print(f"   - برچسب نمونه اول: {sample['label'].item()}")
        
        # ایجاد دیتاست کاربر (اگر ویژگی‌های کاربر موجود باشد)
        print(f"\n2. ایجاد UserDataset...")
        
        # ایجاد pol_df نمونه اگر وجود ندارد
        if 'pol_df' not in locals():
            pol_df = pd.DataFrame({
                'user_name': ['sample_pol1', 'sample_pol2'],
                'party_name': ['Republican', 'Democrat']
            })
        
        user_dataset = UserDataset(
            dataset_df=labelled_data,
            news_df=news_df,
            pol_df=pol_df,
            label_col=label_col
        )
        
        print(f"   ✅ UserDataset ایجاد شد")
        print(f"   - تعداد نمونه‌ها: {len(user_dataset)}")
        
        # تست یک نمونه
        user_sample = user_dataset[0]
        print(f"   - شکل ویژگی‌های کاربر: {user_sample['feats'].shape}")
        
        # ایجاد DataLoader برای تست
        print(f"\n3. ایجاد DataLoader...")
        batch_size = 32
        text_dataloader = DataLoader(
            text_dataset, 
            batch_size=batch_size, 
            shuffle=True
        )
        
        print(f"   ✅ DataLoader ایجاد شد")
        print(f"   - اندازه batch: {batch_size}")
        print(f"   - تعداد batchها: {len(text_dataloader)}")
        
        # تست یک batch
        for batch in text_dataloader:
            print(f"   - شکل batch ویژگی‌ها: {batch['feats'].shape}")
            print(f"   - شکل batch برچسب‌ها: {batch['label'].shape}")
            break
        
        datasets_created = True
        
    except Exception as e:
        print(f"❌ خطا در ایجاد دیتاست‌ها: {e}")
        datasets_created = False
else:
    print("⚠️ ایجاد دیتاست‌ها امکان‌پذیر نیست")
    datasets_created = False


بخش 8: ایجاد دیتاست‌های PyTorch برای آموزش مدل
⚠️ ایجاد دیتاست‌ها امکان‌پذیر نیست


In [17]:
# ============================================================================
# بخش 9: ایجاد و تست مدل‌های شبکه عصبی
# ============================================================================

print("\n" + "="*60)
print("بخش 9: ایجاد و تست مدل‌های شبکه عصبی")
print("="*60)

if loaded_modules['models'] and datasets_created:
    print("ایجاد و تست مدل‌های شبکه عصبی...")
    
    try:
        # 9.1 ایجاد مدل متن
        print("\n1. ایجاد TextNetwork...")
        text_model = TextNetwork(
            input_shape=50,
            lh=128,
            freeze_bert=True,
            dropout=0.1,
            pre_trained="cardiff",
            pooling="mean"
        )
        
        print(f"   ✅ TextNetwork ایجاد شد")
        print(f"   - پارامترهای مدل: {text_model.params_}")
        
        # 9.2 ایجاد مدل کاربر
        print("\n2. ایجاد UserNetwork...")
        
        # تعیین اندازه ورودی بر اساس ویژگی‌های کاربر
        if 'user_dataset' in locals():
            input_shape = user_dataset[0]['feats'].shape[0]
        else:
            input_shape = 100  # مقدار پیش‌فرض
        
        user_model = UserNetwork(
            input_shape=input_shape,
            l1_h=512,
            dropout=0.1,
            activation="relu"
        )
        
        print(f"   ✅ UserNetwork ایجاد شد")
        print(f"   - اندازه ورودی: {input_shape}")
        print(f"   - پارامترهای مدل: {user_model.params_}")
        
        # 9.3 ایجاد مدل ترکیبی
        print("\n3. ایجاد CombinedNetwork...")
        combined_model = CombinedNetwork(
            user_input_shape=input_shape,
            text_lh=128,
            user_lh=64,
            freeze_bert=True,
            dropout=0.1,
            pre_trained="cardiff",
            pooling="mean",
            user_activation="relu",
            text_activation="relu"
        )
        
        print(f"   ✅ CombinedNetwork ایجاد شد")
        print(f"   - پارامترهای مدل: {combined_model.params_}")
        
        # 9.4 تست forward pass
        print("\n4. تست forward pass مدل‌ها...")
        
        # تست TextNetwork
        if 'text_dataloader' in locals():
            for batch in text_dataloader:
                test_input = batch['feats']
                print(f"   - ورودی تست TextNetwork: {test_input.shape}")
                
                # انتقال به دستگاه مناسب
                test_input = test_input.to(device)
                text_model = text_model.to(device)
                
                # forward pass
                with torch.no_grad():
                    hidden, output = text_model(test_input)
                    print(f"   - خروجی TextNetwork: {output.shape}")
                    print(f"   - hidden layer: {hidden.shape}")
                break
        
        models_created = True
        
    except Exception as e:
        print(f"❌ خطا در ایجاد مدل‌ها: {e}")
        models_created = False
else:
    print("⚠️ ایجاد مدل‌ها امکان‌پذیر نیست")
    models_created = False


بخش 9: ایجاد و تست مدل‌های شبکه عصبی
⚠️ ایجاد مدل‌ها امکان‌پذیر نیست


In [18]:
# ============================================================================
# بخش 10: تقسیم داده و آماده‌سازی برای آموزش
# ============================================================================

print("\n" + "="*60)
print("بخش 10: تقسیم داده و آماده‌سازی برای آموزش")
print("="*60)

if labelled_data is not None:
    print("تقسیم داده‌ها به مجموعه‌های train و validation...")
    
    try:
        # تعیین ستون برچسب
        if 'keyword_label' in labelled_data.columns:
            label_col = 'keyword_label'
        elif 'user_label' in labelled_data.columns:
            label_col = 'user_label'
        elif 'sample_label' in labelled_data.columns:
            label_col = 'sample_label'
        else:
            # ایجاد برچسب‌های تصادفی برای نمونه
            labelled_data['train_label'] = np.random.choice([0, 1], len(labelled_data))
            label_col = 'train_label'
        
        # فیلتر کردن نمونه‌های بدون برچسب
        valid_data = labelled_data[labelled_data[label_col] != -1].copy()
        
        if len(valid_data) > 0:
            print(f"تعداد نمونه‌های معتبر برای آموزش: {len(valid_data)}")
            
            # تقسیم داده‌ها
            train_df, val_df = train_test_split(
                valid_data,
                test_size=0.2,
                random_state=42,
                stratify=valid_data[label_col] if len(valid_data[label_col].unique()) > 1 else None
            )
            
            print(f"   - داده‌های آموزش: {len(train_df)} نمونه")
            print(f"   - داده‌های اعتبارسنجی: {len(val_df)} نمونه")
            
            # نمایش توزیع برچسب‌ها
            print(f"\nتوزیع برچسب‌ها در مجموعه آموزش:")
            train_counts = train_df[label_col].value_counts()
            for label, count in train_counts.items():
                print(f"   برچسب {label}: {count} ({count/len(train_df)*100:.1f}%)")
            
            print(f"\nتوزیع برچسب‌ها در مجموعه اعتبارسنجی:")
            val_counts = val_df[label_col].value_counts()
            for label, count in val_counts.items():
                print(f"   برچسب {label}: {count} ({count/len(val_df)*100:.1f}%)")
            
            data_split = True
            
        else:
            print("⚠️ هیچ نمونه معتبری با برچسب مناسب یافت نشد")
            data_split = False
            
    except Exception as e:
        print(f"❌ خطا در تقسیم داده‌ها: {e}")
        data_split = False
else:
    print("⚠️ تقسیم داده‌ها امکان‌پذیر نیست")
    data_split = False


بخش 10: تقسیم داده و آماده‌سازی برای آموزش
⚠️ تقسیم داده‌ها امکان‌پذیر نیست


In [19]:
# ============================================================================
# بخش 11: نمونه‌ای از آموزش مدل (آموزش کوتاه)
# ============================================================================

print("\n" + "="*60)
print("بخش 11: نمونه‌ای از آموزش مدل (آموزش کوتاه)")
print("="*60)

if (loaded_modules['model_utils'] and loaded_modules['torch_datasets'] and 
    data_split and 'train_df' in locals() and 'val_df' in locals()):
    
    print("اجرای یک نمونه آموزش کوتاه برای نمایش فرآیند...")
    
    try:
        # انتخاب زیرمجموعه کوچک برای نمونه آموزش سریع
        sample_train_size = min(200, len(train_df))
        sample_val_size = min(50, len(val_df))
        
        sample_train = train_df.head(sample_train_size).copy()
        sample_val = val_df.head(sample_val_size).copy()
        
        print(f"اندازه مجموعه‌های نمونه برای آموزش کوتاه:")
        print(f"   - آموزش: {sample_train_size} نمونه")
        print(f"   - اعتبارسنجی: {sample_val_size} نمونه")
        
        # تعیین ستون برچسب
        if 'keyword_label' in sample_train.columns:
            label_col = 'keyword_label'
        elif 'user_label' in sample_train.columns:
            label_col = 'user_label'
        else:
            label_col = 'sample_label' if 'sample_label' in sample_train.columns else 'train_label'
        
        # ایجاد دیتاست‌ها
        print("\nایجاد دیتاست‌های نمونه...")
        
        # ایجاد دیتاست متن
        train_text_dataset = TextDataset(
            dataset_df=sample_train,
            label_col=label_col,
            pre_trained="cardiff",
            max_length=50
        )
        
        val_text_dataset = TextDataset(
            dataset_df=sample_val,
            label_col=label_col,
            pre_trained="cardiff",
            max_length=50
        )
        
        print(f"   ✅ دیتاست‌های متن ایجاد شدند")
        
        # ایجاد مدل
        print("\nایجاد مدل برای آموزش نمونه...")
        sample_model = TextNetwork(
            input_shape=50,
            lh=64,  # کوچکتر برای آموزش سریع
            freeze_bert=True,
            dropout=0.1,
            pre_trained="cardiff",
            pooling="mean"
        )
        
        print(f"   ✅ مدل نمونه ایجاد شد")
        
        # پارامترهای آموزش
        lr = 0.001
        epochs = 3  # فقط 3 دوره برای نمونه
        batch_size = 16
        
        print(f"\nپارامترهای آموزش نمونه:")
        print(f"   - نرخ یادگیری: {lr}")
        print(f"   - تعداد دوره‌ها: {epochs}")
        print(f"   - اندازه batch: {batch_size}")
        
        # آموزش مدل (کامنت شده - در صورت نیاز می‌توانید آن را فعال کنید)
        print("\n⚠️ آموزش مدل به صورت کامنت شده است (برای جلوگیری از اجرای طولانی)")
        print("برای اجرای آموزش، کامنت خط زیر را بردارید:")
        print("""
        # آموزش مدل
        best_model, train_losses, val_losses, train_accs, val_accs = train_network(
            train_dataset=train_text_dataset,
            val_dataset=val_text_dataset,
            model=sample_model,
            lr=lr,
            epochs=epochs,
            batch_size=batch_size,
            seed=42,
            plot=True,
            patience=2,
            user=False,
            device_ids=[0] if torch.cuda.is_available() else []
        )
        """)
        
        # به جای آموزش واقعی، یک ساختار نمونه نشان می‌دهیم
        print("\nساختار خروجی آموزش:")
        print("""
        best_model: مدل آموزش دیده بهترین
        train_losses: لیست lossهای آموزش در هر دوره
        val_losses: لیست lossهای اعتبارسنجی در هر دوره  
        train_accs: لیست دقت‌های آموزش در هر دوره
        val_accs: لیست دقت‌های اعتبارسنجی در هر دوره
        """)
        
        training_demo = True
        
    except Exception as e:
        print(f"❌ خطا در نمونه آموزش: {e}")
        training_demo = False
else:
    print("⚠️ نمایش نمونه آموزش امکان‌پذیر نیست")
    training_demo = False


بخش 11: نمونه‌ای از آموزش مدل (آموزش کوتاه)
⚠️ نمایش نمونه آموزش امکان‌پذیر نیست


In [20]:
# ============================================================================
# بخش 12: تجسم و تحلیل (با داده‌های نمونه)
# ============================================================================

print("\n" + "="*60)
print("بخش 12: تجسم و تحلیل نتایج")
print("="*60)

if loaded_modules['plot_utils'] and labelled_data is not None:
    print("نمایش نمونه‌ای از توابع تجسم...")
    
    try:
        # ایجاد داده‌های نمونه برای تجسم
        print("\nایجاد داده‌های نمونه برای توابع تجسم...")
        
        # برای plot_distrust_ratios_custom نیاز به ستون‌های خاصی داریم
        viz_data = labelled_data.copy()
        
        # اگر ستون model_preds وجود ندارد، ایجاد می‌کنیم
        if 'model_preds' not in viz_data.columns:
            print("ایجاد ستون‌های نمونه برای تجسم...")
            viz_data['model_preds'] = np.random.choice([0, 1], len(viz_data))
            viz_data['avg_stance'] = np.random.uniform(-3, 3, len(viz_data))
            viz_data['direct_retweet'] = np.random.choice([True, False], len(viz_data))
        
        # نمایش نمونه‌ای از توابع تجسم (کامنت شده)
        print("\nتوابع تجسم موجود:")
        print("1. plot_distrust_ratios_custom(): رسم نسبت‌های عدم اعتماد")
        print("2. plot_stance_entropy(): رسم آنتروپی موضع")
        print("3. plot_roc_curves(): رسم منحنی‌های ROC")
        print("4. criticism_by_time(): تحلیل انتقاد بر اساس زمان")
        
        print("\n⚠️ توابع تجسم به صورت کامنت شده‌اند (نیاز به داده‌های کامل دارند)")
        print("برای اجرای آنها، کامنت خطوط زیر را بردارید:")
        print("""
        # مثال استفاده از plot_stance_entropy
        # plot_stance_entropy(viz_data)
        
        # مثال استفاده از plot_distrust_ratios_custom  
        # plot_distrust_ratios_custom(viz_data, bin_ranges=[(-3,-2),(-2,-1),(-1,1),(1,2),(2,3)])
        """)
        
        # ایجاد یک نمودار ساده برای نمایش
        print("\nایجاد یک نمودار نمونه ساده...")
        
        if 'matched_partisans' in viz_data.columns:
            # هیستوگرام توزیع موضع‌های حزبی
            plt.figure(figsize=(10, 6))
            plt.hist(viz_data['matched_partisans'], bins=20, alpha=0.7, color='steelblue', edgecolor='black')
            plt.xlabel('Partisan Score', fontsize=12)
            plt.ylabel('Frequency', fontsize=12)
            plt.title('Distribution of Partisan Scores in Sample Data', fontsize=14)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            
            # ذخیره نمودار
            plot_path = os.path.join(current_dir, 'sample_plot.png')
            plt.savefig(plot_path, dpi=300, bbox_inches='tight')
            plt.show()
            print(f"✅ نمودار نمونه در {plot_path} ذخیره شد")
        
        visualization_demo = True
        
    except Exception as e:
        print(f"❌ خطا در ایجاد نمونه تجسم: {e}")
        visualization_demo = False
else:
    print("⚠️ نمایش توابع تجسم امکان‌پذیر نیست")
    visualization_demo = False


بخش 12: تجسم و تحلیل نتایج
⚠️ نمایش توابع تجسم امکان‌پذیر نیست


In [21]:
# ============================================================================
# بخش 13: نتیجه‌گیری و خلاصه
# ============================================================================

print("\n" + "="*60)
print("بخش 13: نتیجه‌گیری و خلاصه اجرا")
print("="*60)

print("""
خلاصه اجرای notebook:
=====================

✅ بخش‌های با موفقیت اجرا شده:
""")

# لیست بخش‌های موفق
success_sections = []

if news_df is not None:
    success_sections.append("بارگذاری داده‌های خبری")
if annotated_df is not None:
    success_sections.append("بارگذاری داده‌های حاشیه‌نویسی شده")
if analysis_df is not None:
    success_sections.append("بارگذاری داده‌های تحلیل")

# بررسی ماژول‌های بارگذاری شده
for module_name, status in loaded_modules.items():
    if status:
        success_sections.append(f"بارگذاری {module_name}")

if preprocessed_data is not None:
    success_sections.append("پیش‌پردازش داده‌ها")
if labelled_data is not None:
    success_sections.append("برچسب‌زنی داده‌ها")
if datasets_created:
    success_sections.append("ایجاد دیتاست‌های PyTorch")
if models_created:
    success_sections.append("ایجاد مدل‌های شبکه عصبی")
if data_split:
    success_sections.append("تقسیم داده‌ها")
if visualization_demo:
    success_sections.append("نمایش توابع تجسم")



بخش 13: نتیجه‌گیری و خلاصه اجرا

خلاصه اجرای notebook:

✅ بخش‌های با موفقیت اجرا شده:



In [22]:
# نمایش بخش‌های موفق
for i, section in enumerate(success_sections, 1):
    print(f"   {i}. {section}")

print(f"\n📊 آمار کلی اجرا:")
print(f"   - تعداد کل بخش‌ها: 13")
print(f"   - بخش‌های با موفقیت اجرا شده: {len(success_sections)}")
print(f"   - نرخ موفقیت: {len(success_sections)/13*100:.1f}%")

print("""
📋 گام‌های بعدی برای اجرای کامل پروژه:
======================================

1. آماده‌سازی داده‌های کامل:
   - استفاده از تمام داده‌های موجود در data_for_analysis_anonymized.jsonl
   - تطبیق ساختار داده‌ها با انتظارات کد

2. آموزش مدل‌ها:
   - آموزش TextNetwork با داده‌های متن
   - آموزش UserNetwork با ویژگی‌های کاربر
   - آموزش CombinedNetwork با ترکیب هر دو

3. ارزیابی مدل‌ها:
   - محاسبه معیارهای ارزیابی (دقت، recall، F1-score، AUC)
   - مقایسه عملکرد مدل‌های مختلف

4. تحلیل نتایج:
   - تجسم توزیع پیش‌بینی‌ها
   - تحلیل خطاهای مدل
   - تفسیر ویژگی‌های مهم

5. استنتاج و deployment:
   - ایجاد خط لوله پیش‌بینی برای داده‌های جدید
   - بهینه‌سازی مدل‌ها برای اجرای واقعی

🔧 نکات فنی:
===========

- برای آموزش کامل نیاز به GPU با حافظه کافی دارید
- زمان آموزش مدل‌های BERT-based می‌تواند طولانی باشد
- پیش‌پردازش داده‌های بزرگ نیاز به مدیریت حافظه دارد

📁 فایل‌های ایجاد شده در این اجرا:
===============================
""")

   1. بارگذاری feature_extractors
   2. بارگذاری analysis_utils

📊 آمار کلی اجرا:
   - تعداد کل بخش‌ها: 13
   - بخش‌های با موفقیت اجرا شده: 2
   - نرخ موفقیت: 15.4%

📋 گام‌های بعدی برای اجرای کامل پروژه:

1. آماده‌سازی داده‌های کامل:
   - استفاده از تمام داده‌های موجود در data_for_analysis_anonymized.jsonl
   - تطبیق ساختار داده‌ها با انتظارات کد

2. آموزش مدل‌ها:
   - آموزش TextNetwork با داده‌های متن
   - آموزش UserNetwork با ویژگی‌های کاربر
   - آموزش CombinedNetwork با ترکیب هر دو

3. ارزیابی مدل‌ها:
   - محاسبه معیارهای ارزیابی (دقت، recall، F1-score، AUC)
   - مقایسه عملکرد مدل‌های مختلف

4. تحلیل نتایج:
   - تجسم توزیع پیش‌بینی‌ها
   - تحلیل خطاهای مدل
   - تفسیر ویژگی‌های مهم

5. استنتاج و deployment:
   - ایجاد خط لوله پیش‌بینی برای داده‌های جدید
   - بهینه‌سازی مدل‌ها برای اجرای واقعی

🔧 نکات فنی:

- برای آموزش کامل نیاز به GPU با حافظه کافی دارید
- زمان آموزش مدل‌های BERT-based می‌تواند طولانی باشد
- پیش‌پردازش داده‌های بزرگ نیاز به مدیریت حافظه دارد

📁 فایل‌های ایجاد شده در

In [23]:
# لیست فایل‌های ایجاد شده
created_files = []
if 'preprocessed_path' in locals() and os.path.exists(preprocessed_path):
    created_files.append(preprocessed_path)
if 'plot_path' in locals() and os.path.exists(plot_path):
    created_files.append(plot_path)

for file in created_files:
    file_size = os.path.getsize(file) / 1024  # حجم به کیلوبایت
    print(f"   - {os.path.basename(file)} ({file_size:.1f} KB)")

print("\n" + "="*60)
print("🎉 اجرای notebook با موفقیت به پایان رسید!")
print("="*60)

# نمایش پیام نهایی
print("""
برای اجرای کامل پروژه:
1. کامنت‌های بخش آموزش را بردارید
2. از داده‌های کامل استفاده کنید
3. پارامترهای مدل را تنظیم کنید

موفق باشید! ✨
""")


🎉 اجرای notebook با موفقیت به پایان رسید!

برای اجرای کامل پروژه:
1. کامنت‌های بخش آموزش را بردارید
2. از داده‌های کامل استفاده کنید
3. پارامترهای مدل را تنظیم کنید

موفق باشید! ✨

